## Turing Machine Simulator

<pre>
CSC 427, Semester 232
Burton Rosenberg
University of Miami
8 march 2023

Copyright 2023 (c) Burton Rosenberg. All rights reserved.
</pre>




### Introduction

The file `turing_machine_simulator.py` **([code](https://github.com/csc427-242/sketches/blob/main/turing_machine_sim.py))** has two classes, `Turing Machine` and `MachineParser`.

The simulator class instantiates an object containing the rules, the states and alphabets, 
and can run and depict the run. To ease programming, the parser class creates the Turing Machine
and writes its program from a description given in this [syntax description](https://github.com/csc427-242/sketches/blob/main/TM_Syntax.md).



### A Simple RE recognizer.

An example for the syntax is the following program recognizing a+b+c+. 

Creating a program for a Regular language is easy. Once a Deterministic Finite Automata is calculated from the language, this can exactly be the turing program. The program runs in linear time, never backs up, and never writes to the tape. In fact, these restrictions are sufficient to claim, conversely, the lanuage accepted is regular. See <a href="https://arxiv.org/abs/cs/0310046">Theory of One Tape Linear Time Turing Machines</a>, Tadaki, Yamakami, Lin.

_Note:_ The paramters to compute_tm are,
- the string to decide,
- a parameter giving maximum number of steps permitted to the computation
- and an optional verbose parameter, ether none, explain, or verbose.



In [1]:
tm_abc= """# a TM program to recognize a+b+c+
# corrected 29 march 2024

accept: A
reject: R
start: q0

state: q0
    a a r qa  # must have an a
state: qa    
    a a r qa  # loop over further a's
    b b r q1  # until a b
              # note: missing transtions cause the machine to reject (c or _)
state: q1
    b b r qb  # must have a b
state: qb
    b b r qb  # continue over further b's
    c c r q2  # until a c
state: q2
    c c r qc  # must have a c
state: qc
    c c r qc  # continue over further c's
    _ _ r A   # until the end of the tape

"""

# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 100

from turing_machine_sim import *

tm = MachineParser.create_from_description(tm_abc)
tm.compute_tm('aabbcc', max_steps, verbose=verbose)
tm.compute_tm('abca', max_steps, verbose=verbose)
tm.compute_tm('bc', max_steps, verbose=verbose)


0 [q0]	[a]abbcc_
1 [qa]	a[a]bbcc_
2 [qa]	aa[b]bcc_
3 [q1]	aab[b]cc_
4 [qb]	aabb[c]c_
5 [q2]	aabbc[c]_
6 [qc]	aabbcc[_]
7 [A]	aabbcc_[_]
accept (ok)
0 [q0]	[a]bca_
1 [qa]	a[b]ca_
2 [q1]	ab[c]a_
reject (transition missing)
0 [q0]	[b]c_
reject (transition missing)


False

## The copy language

Accept strings of the form `w&w` where `w` is a string over `{a,b}^*`. This language was at this point in the class, the apex predator. It is not Context Free, the more powerful of the two language classes studied.

This uses the zig-zag marking approach.

In [2]:
tm_m1 = """# w&w example m1 in the textbook

accept: A
reject: R
start:  S

state: S
    a x r looka1 # mark off the a, now look for a matching a
    b x r lookb1 # mark off the b, now look for a matching b
    & : r end1   # the left w is all marked off
    
state: looka1
    : : r looka1 # skip over a or b
    & : r looka2 # until a & is found
    _ : r R      # in the case there is no &, abort
    
state: looka2
    x x r looka2 # skip over x 
    a x l back1  # until the matching a is found
    
state: lookb1
    : : r lookb1 # skip over a or b  
    & : r lookb2 # until a & is found
    _ : r R      # in the case there is no &, abort
    
state: lookb2
    x x r lookb2 # skip over x
    b x l back1  # until the matching b is found

state: back1
    x x l back1 # skip over x
    & & l back2 # until an & is found
    
state: back2
    : : l back2 # skip over a or b
    x x r S     # until an x is found

state: end1
    x x r end1 # skip over x
    _ _ r A    # until a blank is found

"""


# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 100

tm = MachineParser.create_from_description(tm_m1)
tm.compute_tm('aab&aab', max_steps, verbose=verbose)
print('')
tm.compute_tm('aab&abb', max_steps, verbose=verbose)
print('')
tm.compute_tm('&', max_steps, verbose=verbose)
print('')
tm.compute_tm('', max_steps, verbose=verbose)

0 [S]	[a]ab&aab_
1 [looka1]	x[a]b&aab_
2 [looka1]	xa[b]&aab_
3 [looka1]	xab[&]aab_
4 [looka2]	xab&[a]ab_
5 [back1]	xab[&]xab_
6 [back2]	xa[b]&xab_
7 [back2]	x[a]b&xab_
8 [back2]	[x]ab&xab_
9 [S]	x[a]b&xab_
10 [looka1]	xx[b]&xab_
11 [looka1]	xxb[&]xab_
12 [looka2]	xxb&[x]ab_
13 [looka2]	xxb&x[a]b_
14 [back1]	xxb&[x]xb_
15 [back1]	xxb[&]xxb_
16 [back2]	xx[b]&xxb_
17 [back2]	x[x]b&xxb_
18 [S]	xx[b]&xxb_
19 [lookb1]	xxx[&]xxb_
20 [lookb2]	xxx&[x]xb_
21 [lookb2]	xxx&x[x]b_
22 [lookb2]	xxx&xx[b]_
23 [back1]	xxx&x[x]x_
24 [back1]	xxx&[x]xx_
25 [back1]	xxx[&]xxx_
26 [back2]	xx[x]&xxx_
27 [S]	xxx[&]xxx_
28 [end1]	xxx&[x]xx_
29 [end1]	xxx&x[x]x_
30 [end1]	xxx&xx[x]_
31 [end1]	xxx&xxx[_]
32 [A]	xxx&xxx_[_]
accept (ok)

0 [S]	[a]ab&abb_
1 [looka1]	x[a]b&abb_
2 [looka1]	xa[b]&abb_
3 [looka1]	xab[&]abb_
4 [looka2]	xab&[a]bb_
5 [back1]	xab[&]xbb_
6 [back2]	xa[b]&xbb_
7 [back2]	x[a]b&xbb_
8 [back2]	[x]ab&xbb_
9 [S]	x[a]b&xbb_
10 [looka1]	xx[b]&xbb_
11 [looka1]	xxb[&]xbb_
12 [looka2]	xxb&[x]bb_
13 [loo

False

## Pure powers of two

This machine accepts strings of all 0's, whose length is a pure power of two. 

This is machine M2 in the textbook. It works in $O(n\log n)$ time by repeatedly halving
the length of the string, unless there is a remainer. 

In [3]:
tm_m2 = """# M2 strings 0^k for a pure power of two

accept: qa
reject: qr
start:  q1

state: q1
    0 _ r q2
    _ : r qr
    x : r qr
    
state: q2
    x : r q2
    0 x r q3
    _ : r qa
    
state: q3
    x : r q3
    0 : r q4
    _ : l q5
    
state: q4
    x : r q4
    0 x r q3
    _ : r qr
    
state: q5
    0 : l q5
    x : l q5
    _ : r q2

"""

# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 200

tm = MachineParser.create_from_description(tm_m2)

print('zero')
tm.compute_tm('', max_steps, verbose=verbose)
print('\none')
tm.compute_tm('0', max_steps, verbose=verbose)
print('\ntwo')
tm.compute_tm('00', max_steps, verbose=verbose)
print('\nthree')
tm.compute_tm('000', max_steps, verbose=verbose)
print('\nfour')
tm.compute_tm('0000', max_steps, verbose=verbose)
print('\nfive')
tm.compute_tm('00000', max_steps, verbose=verbose)


zero
0 [q1]	[_]
1 [qr]	_[_]
reject (ok)

one
0 [q1]	[0]_
1 [q2]	_[_]
2 [qa]	__[_]
accept (ok)

two
0 [q1]	[0]0_
1 [q2]	_[0]_
2 [q3]	_x[_]
3 [q5]	_[x]_
4 [q5]	[_]x_
5 [q2]	_[x]_
6 [q2]	_x[_]
7 [qa]	_x_[_]
accept (ok)

three
0 [q1]	[0]00_
1 [q2]	_[0]0_
2 [q3]	_x[0]_
3 [q4]	_x0[_]
4 [qr]	_x0_[_]
reject (ok)

four
0 [q1]	[0]000_
1 [q2]	_[0]00_
2 [q3]	_x[0]0_
3 [q4]	_x0[0]_
4 [q3]	_x0x[_]
5 [q5]	_x0[x]_
6 [q5]	_x[0]x_
7 [q5]	_[x]0x_
8 [q5]	[_]x0x_
9 [q2]	_[x]0x_
10 [q2]	_x[0]x_
11 [q3]	_xx[x]_
12 [q3]	_xxx[_]
13 [q5]	_xx[x]_
14 [q5]	_x[x]x_
15 [q5]	_[x]xx_
16 [q5]	[_]xxx_
17 [q2]	_[x]xx_
18 [q2]	_x[x]x_
19 [q2]	_xx[x]_
20 [q2]	_xxx[_]
21 [qa]	_xxx_[_]
accept (ok)

five
0 [q1]	[0]0000_
1 [q2]	_[0]000_
2 [q3]	_x[0]00_
3 [q4]	_x0[0]0_
4 [q3]	_x0x[0]_
5 [q4]	_x0x0[_]
6 [qr]	_x0x0_[_]
reject (ok)


False

### Erasing the Tape

Accept everything, while erasing the tape leaving the head at the start of the tape.

A Turing Machine opens up another aspect of computation. So far, the algorithms in this
class have only a boolean result: true or false for membership of an element in a set.
(In fact, membership in a subset of a set, as we always assume the input is of proper form.)

This example intrduces another meaning for computation. It is the more common meaning of
"doing something" &mdash; in this case, the TM erases the tape.

In [4]:
tm_erase = """# erase the tape

accept: A
reject: R
start: q0

state: q0    # write a tape endmarker
    : $ R q1
    
state: q1
    : : r q1 # more right not replacing symbols
    _ _ l q2 # begin the erase while returning
    
state: q2
    : _ l q2 # erase symbols
    $ _ N A  # stop at the endmarker
"""


# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 100

tm = MachineParser.create_from_description(tm_erase)
tm.compute_tm('abc', max_steps, verbose=verbose)


0 [q0]	[a]bc_
1 [q1]	$[b]c_
2 [q1]	$b[c]_
3 [q1]	$bc[_]
4 [q2]	$b[c]_
5 [q2]	$[b]__
6 [q2]	[$]___
7 [A]	[_]___
accept (ok)


True

### END